### Load CEX data and clean up
I have access to Binance price data from 2024-01-29 00:00:00.066000 UTC. Will use PENDLEUSDT and WETHUSDT pairs to derive PENDLE-ETH price. Think it's fine to use Binance ETH as essentially the same as WETH because it's more about the relative price change that is important for this analysis.

I need to do the following work:
1. **Pull PENDLEUSDT and ETHUSDT binance data.**
2. **Use Polars to join data on second to produce PENDLEWETH pricing with second-level granularity.** We use Google BQ, and I don't think I can join on second-level data in GoogleBQ. The plan is to join the data in Polars with second-level granularity.

Please note that I'm developing in a haphazard and suboptimal way right now then i'll go back and clean everything up (maybe)

In [11]:
# dependencies

# !pip3 install polars
# !pip3 install plotly

import polars as pl
import plotly.express as px

In [12]:
# see CEX_data_pull.sql. Looks like 8m rows and 400 MB.
# I've never worked with a dataset this large before, so pretty exciting.
# Still, not too big as I understand so should be approachable.

nance = pl.read_csv('data/2024.2.19 PENDLEUSDT WETHUSDT.csv')

In [13]:
# convert string to timestamp
nance = nance.with_columns(
    pl.coalesce(
        pl.col('timestamp')
            # note all times UTC
            .str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S%.f UTC', strict=False), # strict = False writes to null
        # need to coalesce two `strptime` to handle errors where a whole second didn't have a decimal at the end.
        pl.col('timestamp')
            .str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S UTC', strict=False)
    )
)
# truncate timestamp to seconds
nance = nance.with_columns(
    pl.col("timestamp")
    .dt.truncate("1s")
)

# pivot nance table into columns per token pair
nance = nance.pivot(
    values='price',
    index='timestamp',
    columns='symbol',
    aggregate_function='mean' 
)

nance.head()

timestamp,ETHUSDT,PENDLEUSDT
datetime[μs],f64,f64
2024-02-04 21:36:59,2308.10375,3.1367
2024-02-04 21:36:58,2308.1,null
2024-02-04 21:36:57,2308.1,null
2024-02-04 21:36:56,2308.11,null
2024-02-04 21:36:55,2308.11,3.136738


In [14]:
# @Dev TODO is there a way to do this without creating a new dataframe?

# create a polars datetime range from min to max
dates = pl.datetime_range(
            nance.select(pl.min('timestamp')).item(),
            nance.select(pl.max('timestamp')).item(),
            interval = '1s',
            closed = 'both',
            # eager evaluate it into a series
            eager = True
        )   

# create a new df with a continuous timeseries
# this will be my master df
df=pl.DataFrame({'timestamp':dates})
df.shape

(1814401, 1)

In [15]:
# join nance data into df on timestamp
df = df.join(
    nance,
    left_on='timestamp',
    right_on='timestamp',
    how = 'left'
)

In [16]:
# generate new column for PENDLE-WETH price
df = df.with_columns(
    (df['PENDLEUSDT'] / df['ETHUSDT'])
     .alias("nance_PENDLE_ETH_px")
).sort('timestamp') # wanted to explicitly sort before filling forward.

df = df.with_columns(pl.col('nance_PENDLE_ETH_px').fill_null(strategy="forward"))

df.head(10)

timestamp,ETHUSDT,PENDLEUSDT,nance_PENDLE_ETH_px
datetime[μs],f64,f64,f64
2024-01-28 23:59:59,2256.9,null,null
2024-01-29 00:00:00,2256.903922,2.2398,0.000992
2024-01-29 00:00:01,2256.905,null,0.000992
2024-01-29 00:00:02,2256.900667,null,0.000992
2024-01-29 00:00:03,2257.187832,2.240142,0.000992
2024-01-29 00:00:04,2257.78,2.2412,0.000993
2024-01-29 00:00:05,2257.782,2.24085,0.000993
2024-01-29 00:00:06,2257.316185,2.2409,0.000993
2024-01-29 00:00:07,2256.457917,null,0.000993


### Load DEX data and join

Now that my CEX data is set, I'm going to clean my DEX data and merge in. I'm using Dune data.

In [17]:
dex = pl.read_csv('data/2024.2.19 DEX PENDLEWETH.csv')

dex = dex.with_columns(
        pl.col('block_time')
            # cast as timestamp
            .str.to_datetime(
                '%Y-%m-%d %H:%M:%S%.f UTC',
                time_unit='us',
                strict=True)
            # truncate to second
            .dt.truncate("1s")
            # rewrite in place?
            .alias('block_time')
    )

# surely there is a way to rename within the above function?
dex = dex.rename({'block_time':'timestamp'}).sort('timestamp')

dex = dex.with_columns(
    # when the trade is buying WETH, the PENDLEWETH price is ratio of bought/sold
    pl.when(
        pl.col('token_bought_symbol').eq('WETH')
    )
    .then(
        (dex['token_bought_amount']/dex['token_sold_amount'])
    )
    # when the trade is buying PENDLE, the PENDLEWETH price is ratio of sold/bought
    .when(
        pl.col('token_bought_symbol').eq('PENDLE')
    )
    .then(
        (dex['token_sold_amount']/dex['token_bought_amount'])
    )
    .alias('DEX_PENDLE_WETH_px')
)

dex.head(10)

timestamp,token_bought_amount,token_bought_symbol,token_sold_amount,token_sold_symbol,tx_hash,pool,DEX_PENDLE_WETH_px
datetime[μs],f64,str,f64,str,str,str,f64
2024-01-28 00:00:07,0.708812,"""PENDLE""",0.000721,"""WETH""","""0xb3eab8104f8e…","""ARB_30bps""",0.001017
2024-01-28 00:00:48,0.007369,"""WETH""",7.288209,"""PENDLE""","""0x5fd688b5d5f7…","""ARB_30bps""",0.001011
2024-01-28 00:01:10,703.661834,"""PENDLE""",0.715776,"""WETH""","""0x6cae821e67ae…","""ARB_30bps""",0.001017
2024-01-28 00:01:14,391.976954,"""PENDLE""",0.398779,"""WETH""","""0x8283c105dc6b…","""ARB_30bps""",0.001017
2024-01-28 00:01:16,268.19037,"""PENDLE""",0.272867,"""WETH""","""0x5ab4f537275c…","""ARB_30bps""",0.001017
2024-01-28 00:01:21,552.964229,"""PENDLE""",0.562717,"""WETH""","""0xdfff2eca7e8d…","""ARB_30bps""",0.001018
2024-01-28 00:01:21,392.124017,"""PENDLE""",0.398994,"""WETH""","""0xf71eb6fc0dac…","""ARB_30bps""",0.001018
2024-01-28 00:01:22,391.996403,"""PENDLE""",0.398957,"""WETH""","""0x2964eba66bb6…","""ARB_30bps""",0.001018
2024-01-28 00:01:23,499.000054,"""PENDLE""",0.507917,"""WETH""","""0x4bd672f7a7f2…","""ARB_30bps""",0.001018


In [18]:
df = df.join(
    dex.select(
        ['timestamp', 'DEX_PENDLE_WETH_px']
    ),
    on = 'timestamp',
    how = 'left'
)


In [19]:
# I measured where was the first non-null value for my DEX dataset and now I truncate to that point.
df = df[47:].with_columns(
        pl.col('DEX_PENDLE_WETH_px')
        .fill_null(strategy="forward")
    )

In [20]:
df.head(10)

timestamp,ETHUSDT,PENDLEUSDT,nance_PENDLE_ETH_px,DEX_PENDLE_WETH_px
datetime[μs],f64,f64,f64,f64
2024-01-29 00:00:46,2254.16,null,0.000993,0.000998
2024-01-29 00:00:47,2254.1675,null,0.000993,0.000998
2024-01-29 00:00:48,2254.16,null,0.000993,0.000992
2024-01-29 00:00:49,2254.156786,null,0.000993,0.000992
2024-01-29 00:00:50,2254.0825,null,0.000993,0.000992
2024-01-29 00:00:51,2254.08,null,0.000993,0.000992
2024-01-29 00:00:52,2254.00575,2.235962,0.000992,0.000992
2024-01-29 00:00:53,2254.108147,2.2359125,0.000992,0.000992
2024-01-29 00:00:54,2254.597778,null,0.000992,0.000992


### Things to clean up/improve:
1. Keep UTC timestamps in both datasets
2. Impute VWAP for prices.